## Importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import ast

## Importing the tables
### Encryption s-box

In [2]:
col_names  = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']
row_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

enc_key_sbox = pd.read_excel('AES tables.xlsx', index_col=0)

enc_key_sbox.columns = col_names ## replacing the column names 
enc_key_sbox.index = row_names    ## replacing the row names

enc_key_sbox.head()

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
0,63,7c,77,7b,f2,6b,6f,c5,30,1,67,2b,fe,d7,ab,76
1,ca,82,c9,7d,fa,59,47,f0,ad,d4,a2,af,9c,a4,72,c0
2,b7,fd,93,26,36,3f,f7,cc,34,a5,e5,f1,71,d8,31,15
3,4,c7,23,c3,18,96,5,9a,7,12,80,e2,eb,27,b2,75
4,9,83,2c,1a,1b,6e,5a,a0,52,3b,d6,b3,29,e3,2f,84


In [3]:
def text_hexadecimal(text): ## all the blocks --> 16bytes
    hex_text   = []
    for i in text: ## character -> ascii (decimal) -> hexa-decimal
        hex_text.append(hex(ord(i))[2:])
    return hex_text ## 16-byte representation of the text

In [4]:
key = 'satishcjisboring'  
hex_key = text_hexadecimal(key)
print("Hexadecimal Representation : ",hex_key)

Hexadecimal Representation :  ['73', '61', '74', '69', '73', '68', '63', '6a', '69', '73', '62', '6f', '72', '69', '6e', '67']


In [5]:
## splitting into 4*4 matrix
hex_key = [hex_key[i:i+4] for i in range(0, len(hex_key), 4)]
hex_key

[['73', '61', '74', '69'],
 ['73', '68', '63', '6a'],
 ['69', '73', '62', '6f'],
 ['72', '69', '6e', '67']]

## Key Generation

In [6]:
def key_sbox(element):
    row_index = element[0]
    col_index  = element[1]
    ans = enc_key_sbox.loc[[row_index],[col_index]][col_index][0]
    return str(ans)

In [7]:
def binaryToDecimal(binary):
    binary1 = binary
    decimal, i, n = 0, 0, 0
    
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
        
    hexadecimal =  hex(decimal)[-1]
    return hexadecimal

In [8]:
def bcd_hexadecimal(bcd_list):
    w4 = []
    
    for i in range(0,len(bcd_list),2):
        temp_str = ' '
        temp_str = binaryToDecimal(int(bcd_list[i]))
        temp_str = temp_str + binaryToDecimal(int(bcd_list[i+1]))
        w4.append(temp_str)
        
    return w4

In [9]:
def HexaDecimaltoBCD(str):
    list1 = []
    for i in range(len(str)):
        decimal = int(str[i], 16)                                ## hexadecimal -> decimal
        binary_num = bin(decimal).replace("0b", "")  # decimal -> binary
        list1.append(binary_num)
            
    ## binary in-terms of 4 bits
    for i in range(len(list1)):
        element = list1[i]
        if len(element)<4:
            diff = 4 - len(element)
            for j in range(diff):
                element = "0" + element
            list1[i] = element
            
    return list1

In [10]:
def Hexaword_BCD(list1):
    bcd = [ HexaDecimaltoBCD(i) for i in list1 ]
    bcd = list(np.concatenate(bcd).flat) ## 2d list to 1d list
    return bcd

In [11]:
## round-constant table 
## key-round and value-hexadecimal
round_constant = {1:'1', 2:'2', 3:'4', 4:'8', 5:'10', 6:'20', 7:'40', 8:'80', 9:'1B', 10:'36'}

In [12]:
def key_generation():
    total_keys = []
    for i in range(1,10):
        last_col = hex_key[-1]
        rot_word = last_col[i:] + last_col[:i]
        subword = [key_sbox(i) for i in rot_word]

        break

In [13]:
hex_key

[['73', '61', '74', '69'],
 ['73', '68', '63', '6a'],
 ['69', '73', '62', '6f'],
 ['72', '69', '6e', '67']]

In [14]:
hex_key[-1]

['72', '69', '6e', '67']

## g function

In [15]:
last_col = hex_key[-1]
rot_word = last_col[1:] + last_col[:1] ## rotating the words
rot_word

['69', '6e', '67', '72']

In [16]:
#enc_key_sbox.loc[[60],[9]]
subword = [key_sbox(i) for i in rot_word]
subword

['f9', '9f', '85', '40']

In [17]:
y1 = Hexaword_BCD(subword)
y1

['1111', '1001', '1001', '1111', '1000', '0101', '0100', '0000']

In [18]:
iteration = 1

element = round_constant[iteration]
initial = HexaDecimaltoBCD(element)
initial_length = len(initial)

if (initial_length == 1):
    between = initial
    temp_1 = [ [0 for j in range(4) for i in range(1)] ]
    last = [ [0 for j in range(4)] for i in range(7) ]
    between.extend(last)
    temp_1.extend(last)
    final = temp_1

else:
    last = [ [0 for j in range(4)] for i in range(6) ]
    first = initial
    first.extend(last)
    
final = [ "".join(list(map(str, i))) for i in final ]
round_list = final

round_list

['0000', '0000', '0000', '0000', '0000', '0000', '0000', '0000']

In [19]:
def XOR(list1, list2):
    def compare(element1, element2):
        ans = []
        for i,j in zip(element1,element2):
            if (i==j):ans.append(0)
            else:ans.append(1)
        return ans

    main_ans = []
    for i in range(len(list1)):
        main_ans.append(compare(list1[i], list2[i]))
    main_ans = [ "".join(list(map(str, i))) for i in main_ans ]
    
    return main_ans

In [20]:
print(y1)
print(round_list)

['1111', '1001', '1001', '1111', '1000', '0101', '0100', '0000']
['0000', '0000', '0000', '0000', '0000', '0000', '0000', '0000']


In [21]:
round_ans = XOR(y1, round_list)
round_ans

['1111', '1001', '1001', '1111', '1000', '0101', '0100', '0000']

In [22]:
col1 = [HexaDecimaltoBCD(i) for i in hex_key[0] ]
col1 = list(np.concatenate(col1).flat)
col1

['0111', '0011', '0110', '0001', '0111', '0100', '0110', '1001']

In [23]:
ans = XOR(col1, round_ans) ## w4 = w0 + g(w3)
ans

['1000', '1010', '1111', '1110', '1111', '0001', '0010', '1001']

In [24]:
w4 = bcd_hexadecimal(ans)
w4

['8a', 'fe', 'f1', '29']

In [25]:
'''
round-1 [w4, w5, w6, w7]
round-2 [w8, w9, w10, w11]
. 
.
round-10 [w40, w41, w42, w43]
'''

'\nround-1 [w4, w5, w6, w7]\nround-2 [w8, w9, w10, w11]\n. \n.\nround-10 [w40, w41, w42, w43]\n'

## key loop (1 to 10)

### w5 = w4 exor w1_before

In [26]:
col1_before = Hexaword_BCD(hex_key[1])
col1 = Hexaword_BCD(w4)  ## w0 exor g(w3)

print("w1 : " ,col1_before)
print("w4 : " ,col1)

col2 = XOR(col1_before, col1)
print("ans : ",col2)

ans = bcd_hexadecimal(col2)
print("w5 : ",ans)

w1 :  ['0111', '0011', '0110', '1000', '0110', '0011', '0110', '1010']
w4 :  ['1000', '1010', '1111', '1110', '1111', '0001', '0010', '1001']
ans :  ['1111', '1001', '1001', '0110', '1001', '0010', '0100', '0011']
w5 :  ['f9', '96', '92', '43']


In [30]:
col1_before

['0111', '0011', '0110', '1000', '0110', '0011', '0110', '1010']

In [31]:
col1

['1000', '1010', '1111', '1110', '1111', '0001', '0010', '1001']

### w6 = w5 exor w2_before

In [27]:
col2_before = Hexaword_BCD(hex_key[2])

print("w2 : " ,col2_before)
print("w5 : " ,col2)

col3 = XOR(col2_before, col2)
print("ans : ",col2)

ans = bcd_hexadecimal(col3)
print("w6 : ",ans)

w2 :  ['0110', '1001', '0111', '0011', '0110', '0010', '0110', '1111']
w5 :  ['1111', '1001', '1001', '0110', '1001', '0010', '0100', '0011']
ans :  ['1111', '1001', '1001', '0110', '1001', '0010', '0100', '0011']
w6 :  ['90', 'e5', 'f0', '2c']


### w7 = w6 exor w3_before

In [28]:
col3_before = Hexaword_BCD(hex_key[3])

print("w3 : " ,col3_before)
print("w5 : " ,col3)

col4 = XOR(col3_before, col3)
print("ans : ",col4)

ans = bcd_hexadecimal(col4)
print("w5 : ",ans)

w3 :  ['0111', '0010', '0110', '1001', '0110', '1110', '0110', '0111']
w5 :  ['1001', '0000', '1110', '0101', '1111', '0000', '0010', '1100']
ans :  ['1110', '0010', '1000', '1100', '1001', '1110', '0100', '1011']
w5 :  ['e2', '8c', '9e', '4b']


In [29]:
for i in range(1,10): ## 10 times running
    ## 1st word = col_before[0] (exor) g(col_before[-1]) 
    
    for j in range(1,4) ## each loop, 3 times running ( remaining 3 words)

SyntaxError: invalid syntax (606241682.py, line 4)

In [ ]:
j = 2

In [ ]:
"col" + str(j) + "_before"

In [ ]:
sbox[2][3]

In [ ]:
'''
total_keys = []
for i in range(1,10):
    last_col = hex_key[-1]
    rot_word = last_col[1:] + last_col[:1]
    subword = [key_sbox(i) for i in rot_word]
'''

In [ ]:
# Python3 program to convert the given
# HexaDecimal number to its equivalent BCD

# Function to convert
# Haxadecimal to BCD
def HexaDecimaltoBCD(str):

	# Iterating through the digits
	for i in range(len(str)):
		
		# Conversion into equivalent BCD
		decimal = int(str[i], 16)
		binary_num = bin(decimal).replace("0b", "")
		print(binary_num)

# Driver code
str = "1B"

# Function call
HexaDecimaltoBCD(str)

# This code is contributed by himanshu77
